# Intro to Music21

Following along with [this playlist](https://www.youtube.com/watch?v=zanx4IS3p24&list=PL1dIy2tm36HUn36wXyfXHauDFKO8LviCO) for starters.

Note that I did this notebook on 2024-03-02 and the most recent video (number 11) was only 2 weeks old, so it's quite possible new videos will pop up later :)

In [ ]:
import subprocess

from music21 import environment

user_settings = environment.UserSettings()

# Find musescore provided via nix flakes
MUSESCORE_EXE = subprocess.check_output(["which", "mscore."]).strip().decode()
MUSESCORE_DIR = MUSESCORE_EXE.removesuffix('/bin/mscore.')
MUSESCORE_APP = MUSESCORE_DIR + "/Applications/mscore.app/"
user_settings["musicxmlPath"] = MUSESCORE_APP
user_settings["musescoreDirectPNGPath"] = MUSESCORE_EXE

# Find lilypond provided via nix flakes
LILYPOND_EXE = subprocess.check_output(["which", "lilypond"]).strip().decode()
LILYPOND_VERSION = subprocess.check_output(["lilypond", "--version"]).strip().decode().split()[2]
user_settings["lilypondPath"] = LILYPOND_EXE
user_settings["lilypondVersion"] = LILYPOND_VERSION

dict(user_settings)

In [ ]:
# Video 1: notes and basics of streams
# https://www.youtube.com/watch?v=zanx4IS3p24&list=PL1dIy2tm36HUn36wXyfXHauDFKO8LviCO
from music21 import (
    note,
    stream,
)

n = note.Note('C4')
print(n)
print(n.name, n.octave, f"frequency: {n.pitch.frequency}")

print(n.quarterLength)
n.quarterLength = 0.5
print(n.quarterLength)

melody = stream.Stream()
melody.append(n)
melody.append(note.Note('D4'))
melody.show("text")
# Use: .show("midi") to play inline, .show() to display inline, .show("musicxml") to send to musescore

In [ ]:
# Video 2: scores, parts, and voices
# https://www.youtube.com/watch?v=Nv6CW2IJHG0&list=PL1dIy2tm36HUn36wXyfXHauDFKO8LviCO&index=2
from music21 import (
    note,
    stream,
)

score = stream.Score()

melody_and_harmony = stream.Part()
bassline = stream.Part()

voice1 = stream.Voice()
voice2 = stream.Voice()

note_names = [
    "C4",
    "D4",
    "E4",
    "F4",
    "G4",
    "A4",
    "B4",
    "C5",
]
for note_name in note_names:
    melody_note = note.Note(note_name)
    harmony_note = melody_note.transpose(-8)
    bass_note = note.Note(note_name)
    bass_note.octave -= 2
    
    voice1.append(melody_note)
    voice2.append(harmony_note)
    bassline.append(harmony_note)

melody_and_harmony.append([voice1, voice2])

score.insert(0, melody_and_harmony)
score.insert(0, bassline)

score.show("text")

In [ ]:
# Video 3: durations
# https://www.youtube.com/watch?v=tNK7XhRY8yM&list=PL1dIy2tm36HUn36wXyfXHauDFKO8LviCO&index=3
from music21 import (
    note,
    stream,
    duration,
    meter,
)

d = duration.Duration(type="quarter", dots=1)
n = note.Note("C4", duration=d)
# n.show()

s = stream.Stream()
# basic note in a time signature; use float instead of type + dots
s.insert(meter.TimeSignature("4/4"))
n0 = note.Note("F4", duration=duration.Duration(1.5))
s.append(n0)
# force a split (tie) instead of a dot
n1 = note.Note("F4", duration=duration.Duration(1.5))
n1 = n1.splitAtQuarterLength(1)
s.append(n1)
# force multiple splits
n2 = note.Note("F4", duration=duration.Duration(1.5))
n2 = n2.splitByQuarterLengths([0.5, 0.25, 0.25, 0.5])
s.append(n2)

# take a look
s.show("text", addEndTimes=True)



In [ ]:
from music21 import (
    note,
    stream,
    duration,
    meter,
)

# insertAndShift is a way to insert notes at specified offsets instead of just appending them
#
# The signature is weird, it takes a list of alternating numbers and notes as inputs.
# More importantly, by default no rests are inserted so it's actually quite hard to see that
# it's even working; I need to learn how to auto-insert rests I guess

s = stream.Stream()

s.insertAndShift(
    0,
    note.Note("C4", duration=duration.Duration(0.5)),
)
s.insertAndShift(
    1.5,
    note.Note("D4", duration=duration.Duration(0.5)),
)
s.show("text", addEndTimes=True)

In [ ]:
# Skipping for now...
#
# Video 4: the corpus + your own files
#    https://www.youtube.com/watch?v=_5GGXZMlKbI&list=PL1dIy2tm36HUn36wXyfXHauDFKO8LviCO&index=4
#
# Video 5: 12-tone rows example
#    https://www.youtube.com/watch?v=_5GGXZMlKbI&list=PL1dIy2tm36HUn36wXyfXHauDFKO8LviCO&index=4

In [ ]:
# Video 6: chords and chordify
#   https://www.youtube.com/watch?v=qy5fc5lJ6U0&list=PL1dIy2tm36HUn36wXyfXHauDFKO8LviCO&index=6
from music21 import (
    note,
    stream,
    duration,
    meter,
    chord,
)
import copy

# You can create with just note name, just numbers, or actual notes
chord0 = chord.Chord(["C5", "E5", "G5"])
chord1 = chord.Chord([0, 4, 7])
chord2 = chord.Chord([note.Note("C5"), note.Note("E5"), note.Note("G5")])

# Combining chords
power_chord_0 = chord.Chord(["C5", "G5"])
power_chord_1 = chord.Chord(["E5", "B5"])

print("power_chord_0 root: ", power_chord_0.root())
print("power_chord_1 root: ", power_chord_1.root())

print("power_chord_1 name: ", power_chord_1.commonName)


major_seven_chord = copy.deepcopy(power_chord_0)
major_seven_chord.add(power_chord_1.pitches)
print("Cma7: ", major_seven_chord)
print("  ...is   7th: ", major_seven_chord.isSeventh())
print("  ...is triad: ", major_seven_chord.isTriad())
print("  ...    name: ", major_seven_chord.commonName)

major_chord = copy.deepcopy(major_seven_chord)
major_chord.remove("B5")
print("Cma: ", major_chord)
print("  ...is   7th: ", major_chord.isSeventh())
print("  ...is triad: ", major_chord.isTriad())
print("  ...    name: ", major_chord.commonName)

print("  ... and some more theoretical stuff ...")
print("  ...   forteClass: ", major_chord.forteClass)
print("  ...   intervalVector: ", major_chord.intervalVector)
print("  ...   getZRelation: ", major_chord.getZRelation())  # what is this? it returns None


major_chord.duration = duration.Duration(1.5)
major_chord.show("text", addEndTimes=True)



In [ ]:
from music21 import corpus
import functools


@functools.lru_cache(1)
def load_movement():
    return corpus.parse("beethoven/opus59no1/movement1.mxl")

In [ ]:
# Chordify: combine many voices into one part as chords

quartet = load_movement()
# quartet.show()

quartet_chords = quartet.chordify()
# quartet_chords.show()

In [ ]:
# Video 7: Complex rhythms using tuplets
#   https://www.youtube.com/watch?v=stviOeKW4Ts&list=PL1dIy2tm36HUn36wXyfXHauDFKO8LviCO&index=7

from music21 import (
    note,
    duration,
    stream,
    meter,
)

# getting tuplet-like behavior manually
n = note.Note(duration=duration.Duration(2.5))
s = stream.Stream()
s.insert(meter.TimeSignature('3/4'))
s.repeatAppend(n, 4)

n.duration = duration.Duration(1/3)
s.repeatAppend(n, 6)
s.show("text", addEndTimes=True)

In [ ]:
from music21 import (
    note,
    duration,
    stream,
    meter,
)

s = stream.Stream()

# using tuplet functionality, which is especially powerful for weird polyrhythms

t = duration.Tuplet(4, 3)
t.tupletNormalShow = "number"

n0 = note.Note(duration=duration.Duration(1))
n0.duration.appendTuplet(t)
s.repeatAppend(n0, 4)

n1 = note.Note(duration=duration.Duration(0.5))
n1.duration.appendTuplet(t)
s.repeatAppend(n1, 4)

s.show("text", addEndTimes=True)

In [ ]:
from music21 import (
    note,
    duration,
    stream,
    meter,
)

s = stream.Stream()

# using tuplets for more normal rhythms
# ... ok this didn't actually work

t = duration.Tuplet(3, 1)
t.tupletNormalShow = "number"

notes = []
for note_name, triplet_length in [("C4", 2/3), ("D4", 1/3), ("E4", 2/3), ("G4", 1/3)]:
    n = note.Note(note_name, duration=duration.Duration(triplet_length))
    n.duration.appendTuplet(t)
    notes.append(n)

s.append(notes)

s.show("text", addEndTimes=True)

In [ ]:
# From around 5m to 10m, video 7 discusses a more complex rhythmic idea that I'm not
# so interested in pursuing right now

# It's a cool example of a real function and a theoretical idea, but maybe not relevant to
# my short term goal of jazz ear training + practice tools

In [ ]:
# Video 8: TinyNotation (loosely a stripped-down lilypond syntax)
#   https://www.youtube.com/watch?v=TSO0fB5Tess&list=PL1dIy2tm36HUn36wXyfXHauDFKO8LviCO&index=8

from music21 import (
    converter
)

# The notation is a little different:
#   - "'" raises an octave, more "'"s raises more octaves (like lily)
#   - Capital lowers and octave, more capitals lowers more octaves (unlike lily)
#   - As in lily, subdivision is after note and persists until otherwise specified
s = converter.parse("""tinyNotation:
2/4
c8 d d# e
A4 g'
c~ c
trip{d8 d d} c4
""")

s.show("text", addEndTimes=True)

In [ ]:
# Video 9: Scale manipulation in music21
#   https://www.youtube.com/watch?v=TSO0fB5Tess&list=PL1dIy2tm36HUn36wXyfXHauDFKO8LviCO&index=9

from music21 import (
    scale,
    stream,
    note,
    converter,
    pitch,
)

sc = scale.HarmonicMinorScale("C")

print(sc.getLeadingTone())
print(sc.getScaleDegreeFromPitch("Eb"))
print(sc.pitchFromDegree(3))

print(sc.transpose(5))
print(sc.transpose('p5'))
print(sc.transpose('M2'))

# note that E4 isn't actually in the scale
print(sc.nextPitch("E4"))
print(sc.nextPitch("E4", 2))
print(sc.nextPitch("E4", -1))
print(sc.nextPitch("E4", -2))

# (-1 will give previous if you are already on a scale pitch)
print(sc.nextPitch("C4", -1))

print(sc.getPitches()[3])  # note that this is different from pitchFromDegree due to 0- vs 1-indexing

In [ ]:
sc = scale.DorianScale("F")
notated_scale = stream.Part([
    note.Note(n) for n in sc.getPitches("F4", "F5", direction=scale.Direction.DESCENDING)
])
notated_scale.show("text")


# Starting at 5m the video discusses creating new scales manually with CyclicalScale
# ... if there are any jazz scales not yet in music21 this would be useful, but I'm skipping for now

In [ ]:
# Using scales to translate melodic patterns
import copy

# The classic Coltrane digital pattern, twice (found, e.g., all over is Giant Steps solo)
coltrane_phrase = converter.parse("""tinyNotation:
c8 d e g
f8 g a c'
""")


def get_scale_distance(p0, p1, scale):
    if p0 > p1:
        multiplier = -1
        p0, p1 = p1, p0
    else:
        multiplier = 1
    notes = scale.getPitches(p0, p1)
    distance = len(notes) - 1
    return multiplier * distance

def translate_phrase(phrase, scale0, scale1, start1):
    notes_old = [
        n
        for m in phrase
        for n in m
        if isinstance(n, note.Note)
    ]
    n0_old = notes_old[0]
    n0_new = copy.deepcopy(n0_old)
    n0_new.pitch = start1
    notes_new = [n0_new]
    last_old_pitch = n0_old.pitch
    last_new_pitch = start1
    print(notes_old)
    for n_old in notes_old[1:]:
        interval = get_scale_distance(last_old_pitch, n_old.pitch, scale0)
        n_new = copy.deepcopy(n_old)
        n_new.pitch = scale1.nextPitch(last_new_pitch, interval)
        notes_new.append(n_new)
        last_old_pitch = n_old.pitch
        last_new_pitch = n_new.pitch
        # somehow the last c5 gets dropped, need to figure that out!
        print(n_old.pitch, "->", n_new.pitch)
    out = stream.Stream()
    out.append(notes_new)
    return out

new_phrase = translate_phrase(
    coltrane_phrase,
    scale.MajorScale("C"),
    scale.MinorScale("D"),
    pitch.Pitch("E4"),
)
new_phrase.show("text")

In [ ]:
# Video 10: Composing with harmonic series
#   https://www.youtube.com/watch?v=XmsF9HmIEis&list=PL1dIy2tm36HUn36wXyfXHauDFKO8LviCO&index=10

from music21 import (
    pitch,
)

fundamental = pitch.Pitch("C4")
print(fundamental.getHarmonic(3))
print(fundamental.getHarmonic(7))  # <- note that this is a fairly precise pitch


# I decided to skip the rest of this video for now; it's interesting, but not very
# relevant to my short-term interests of creating jazz ear training and practice tools

In [ ]:
# Video 11: Composing with random walks
#   https://www.youtube.com/watch?v=QLKnfiY3s84&list=PL1dIy2tm36HUn36wXyfXHauDFKO8LviCO&index=11

from music21 import (
    pitch,
    note,
    stream,
)
import random


def random_walk(count):
    last = 0
    out = [0]
    for i in range(count - 1):
        next = last + random.choice([-1, 1])
        out.append(next)
        last = next
    return out

In [ ]:
def random_chromatic_walk(pitch0, count):
    out = stream.Stream()
    for offset in random_walk(count):
        n = note.Note(pitch0.transpose(offset))
        out.append(n)
    return out

random_stream(pitch.Pitch("G4"), 16).show("midi")

In [ ]:
def offset_from_scale(pitch0, offset, sc):
    if offset == 0:
        return pitch0
    else:
        return sc.nextPitch(pitch0, offset)


def random_scale_walk(pitch0, sc, count):
    out = stream.Stream()
    for offset in random_walk(count):
        n = note.Note(offset_from_scale(pitch0, offset, sc))
        out.append(n)
    return out


random_from_scale(pitch.Pitch("G4"), scale.MajorScale("C"), 16).show("midi")

In [ ]:
# Homework: create a more general walk
# - use a scale
# - allow intervals with weights instead of just +/- 1
# - set boundaries on the walk